In [1]:
import psycopg2
import keys
import cohere

pgconn = psycopg2.connect(
    host="localhost",
    dbname='caroldb',
    user='postgres',
    password=keys.pg_pwd
)
cur = pgconn.cursor()
co=cohere.Client(keys.cohere_key)

k = 5
query_text = "Tell me about the most recent hearing."

# Embed query
query_vec = co.embed(texts=[query_text], model="embed-english-v3.0", input_type = "search_query")

# Search caroldb.pages for k-most similar embeddings, return as page_results
search_query = f"""SELECT * FROM pages ORDER BY embedding <=> {query_vec} LIMIT {k}"""

# Search caroldb.transcripts for all parent_docs from page_results, return as parent_results
#
# Build reference_material:
#   For each doc in parent_results:
#       Add summary
#       Add all page texts from page_results
#
# Invoke model with preamble, query, and refernce material
